In [1]:
import pandas as pd
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from html_table_parser import parser_functions
from urllib.request import urlretrieve
import openpyxl

In [2]:
#pip install html_table_parser

### 타자 날짜별 기록

In [3]:
# 2018-2021
# 생년월일 / 선수이름 / 연도

In [4]:
hitter_info = pd.read_csv('data/hitter_info.csv', index_col=0)
hitter_info = hitter_info.drop_duplicates(['선수코드'], keep='first')

In [5]:
def log_crawler(hitter_info):
    
    year = [2018, 2019, 2020, 2021]
    name = hitter_info['선수명']
    
    hit_log = pd.DataFrame(columns = ['ss', '상대', '결과', '타순', 'P', '선발', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살',
                                 '희타', '희비', '타율', '출루', '장타', 'OPS', '투구', 'avLl', 'RE24', 'WPA', '선수명'])
    print('진행중...')

    for i, j in enumerate(name):
        for k in (year):
            url = 'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name={0}&birth={1}&re=0&se=&da=&year={2}&cv='.format(j, hitter_info['생년월일'][hitter_info.index[i]], k)
            request = requests.get(url)
            soup = BeautifulSoup(request.text, 'html.parser')
            
            try:
                data = soup.find('table', {'class':'table-striped'})
                nm = soup.find('font').text
                table = parser_functions.make2d(data)

                df = pd.DataFrame(data=table[1:], columns=['ss', '상대', '결과', '타순', 'P', '선발', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살',
                              '희타', '희비', '타율', '출루', '장타', 'OPS', '투구', 'avLl', 'RE24', 'WPA'])
                df['선수명'] = nm
                df['연도'] = k

                if(df.shape[0] != 0):
                    hit_log = hit_log.append(df)
                else:
                    continue
            except:
                continue
    print('crawling finished')
    return hit_log

In [6]:
hit_log = log_crawler(hitter_info)

진행중...
crawling finished


In [7]:
hit_log = hit_log.reset_index(drop=True)
hit_log = hit_log.loc[hit_log['타순'] != '타순', :]

In [8]:
hit_log

,ss,상대,결과,타순,P,선발,타수,득점,안타,2타,...,타율,출루,장타,OPS,투구,avLl,RE24,WPA,선수명,연도
0,07-19,@KT,L 7:8,8,PH,0,1,0,1,0,...,1.000,1.000,1.000,2.000,2,2.61,0.95,0.149,김창혁,2018.0
1,07-22,@삼성,L 4:5,8,C,1,3,0,0,0,...,0.250,0.250,0.250,0.500,9,1.03,-1.33,-0.078,김창혁,2018.0
3,09-06,@KT,W 9:2,5,PR,0,2,1,1,1,...,0.333,0.333,0.500,0.833,3,0.02,-0.25,-0.001,김창혁,2018.0
4,09-07,@KT,L 4:12,6,PH,0,1,0,0,0,...,0.286,0.286,0.429,0.714,6,0.00,-0.12,0.000,김창혁,2018.0
5,09-09,@LG,L 5:8,8,PH,0,2,0,0,0,...,0.222,0.222,0.333,0.556,9,0.51,-0.44,-0.026,김창혁,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69634,08-26,삼성,W 4:3,5,3B,1,4,1,1,1,...,0.254,0.379,0.451,0.830,20,1.03,-0.45,-0.029,문보경,2021.0
69635,08-27,삼성,W 3:1,5,3B,1,4,0,0,0,...,0.249,0.372,0.441,0.813,17,1.21,-1.47,-0.140,문보경,2021.0
69636,07-02,@KT,L 1:4,8,1B,1,3,0,1,0,...,0.333,0.500,0.333,0.833,15,1.32,-0.28,0.019,이명기,2021.0
69637,07-04,@KT,L 3:12,8,1B,1,2,0,0,0,...,0.200,0.429,0.200,0.629,18,1.15,-0.84,-0.055,이명기,2021.0


In [9]:
hit_log.to_csv('data/hit_log.csv')

### 경기 로그

In [ ]:
# 날짜 : 2018-03-24 ~ 2021-07-11
# 장소 : 고척돔 / 마산 / 인천문학 / 잠실 / 챔피언스필드 / 대전한밭 / 라이온즈파크 / 부산사직 / 케이티위즈파크
# 시간 : 14 / 17 / 18
# 우천 경기 스킵

In [113]:
url = 'http://www.statiz.co.kr/boxscore.php?opt=5&date=2018-03-25&stadium=%EA%B3%A0%EC%B2%99%EB%8F%94&hour=14'
request = requests.get(url)
soup = BeautifulSoup(request.text, 'html.parser')
            
data = soup.select_one('div.col-sm-12:nth-of-type(1) table.table-striped')
table = parser_functions.make2d(data)
df = pd.DataFrame(data=table[1:], columns=['이닝', '투수', '타자', 'P', '결과', '이전상황', '이후상황', 'LEV', 'REs', 'REa', 'WPe', 'WPa'])
df.shape

(88, 12)

In [108]:
from datetime import datetime, timedelta

def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates

In [117]:
# 날짜 : 2018-03-24 ~ 2021-07-11
# 장소 : 고척돔 / 마산, 창원NC파크 / 인천문학,인천+SSG+랜더스 / 잠실 / 챔피언스필드 / 대전한밭 / 라이온즈파크 / 부산사직 / 케이티위즈파크
# 시간 : 14 / 17 / 18

def gamelog_crawler():
    
    dates = list(date_range("2018-03-24", "2021-07-11"))
    stadium = ['고척돔', '마산', '창원NC파크', '인천문학', '인천+SSG+랜더스', '잠실', '챔피언스필드', '대전한밭', '라이온즈파크', '부산사직', '케이티위즈파크']
    hour = [14, 17, 18]
    
    game_log = pd.DataFrame(columns = ['이닝', '투수', '타자', 'P', '결과', '이전상황', '이후상황', 'LEV', 'REs', 'REa', 'WPe', 'WPa'])
    
    print('진행중...')
    for i in dates:
        for j in stadium:
            for k in hour:
                url = 'http://www.statiz.co.kr/boxscore.php?opt=5&date={0}&stadium={1}&hour={2}'.format(i, j, k)
                request = requests.get(url)
                soup = BeautifulSoup(request.text, 'html.parser')

                try:
                    data = soup.select_one('div.col-sm-12:nth-of-type(1) table.table-striped')
                    table = parser_functions.make2d(data)
                    df = pd.DataFrame(data=table[1:], columns=['이닝', '투수', '타자', 'P', '결과', '이전상황', '이후상황', 'LEV', 'REs', 'REa', 'WPe', 'WPa'])
                    df['날짜'] = i
                    
                    if(df.shape[0] != 0 and df.shape[1] == 13):
                        game_log = game_log.append(df)
                    else:
                        continue         
                except:
                    continue
                
    print('crawling finished')
    return game_log

In [118]:
game_log = gamelog_crawler()

진행중...
crawling finished


In [133]:
game_log

,이닝,투수,타자,P,결과,이전상황,이후상황,LEV,REs,REa,WPe,WPa,날짜
0,1초,로저스,1 이용규,3(2-0),이용규 : 삼진 아웃,무사 0:0,1사 0:0,0.87,0.555,-0.258,52.2%,-0.022,2018-03-24
1,,로저스,2 양성우,4(1-2),중견수 뜬공,1사 0:0,2사 0:0,0.62,0.297,-0.180,53.8%,-0.016,2018-03-24
2,,로저스,3 송광민,1(0-0),좌익수 안타,2사 0:0,2사 1루 0:0,0.40,0.117,0.134,52.6%,0.012,2018-03-24
3,,로저스,4 김태균,3(0-2),우익수 안타,2사 1루 0:0,"2사 1,2루 0:0",0.79,0.251,0.215,50.6%,0.020,2018-03-24
4,,로저스,5 하주석,4(2-1),좌익수 안타,"2사 1,2루 0:0","2사 1,2루 1:0",1.62,0.466,1.000,40.7%,0.099,2018-03-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
730743,,이상민,4 안치홍,6(2-3),볼넷,무사 0:11,무사 1루 0:11,0.00,0.555,0.398,100.0%,0.000,2021-07-11
730744,,,,,대타 : 이대호 -> 김재유,,,,,,,,2021-07-11
730745,,이상민,5 김재유,1(0-0),2루수 내야안타,무사 1루 0:11,"무사 1,2루 0:11",0.00,0.953,0.620,100.0%,0.000,2021-07-11
730746,,이상민,6 한동희,4(2-1),3루수 병살타,"무사 1,2루 0:11",2사 3루 0:11,0.01,1.573,-1.186,100.0%,0.000,2021-07-11


In [120]:
game_log['이닝'] = game_log['이닝'].fillna(method='ffill')
game_log = game_log.reset_index(drop=True)

In [131]:
game_log = game_log.query('이닝 != "정규"')

In [132]:
game_log.to_csv('data/game_log.csv')

In [135]:
game_log.query('날짜 == "2021-05-07"')

,이닝,투수,타자,P,결과,이전상황,이후상황,LEV,REs,REa,WPe,WPa,날짜
670608,1초,원태인,1 정훈,3(1-1),좌익수 뜬공,무사 0:0,1사 0:0,0.87,0.555,-0.258,52.2%,-0.022,2021-05-07
670609,,원태인,2 손아섭,1(0-0),유격수 땅볼,1사 0:0,2사 0:0,0.62,0.297,-0.180,53.8%,-0.016,2021-05-07
670610,,원태인,3 전준우,4(2-1),좌익수 안타,2사 0:0,2사 1루 0:0,0.40,0.117,0.134,52.6%,0.012,2021-05-07
670611,,원태인,4 이대호,4(2-0),중견수 뜬공,2사 1루 0:0,이닝종료 0:0,0.79,0.251,-0.251,54.8%,-0.022,2021-05-07
670612,1말,박세웅,1 김상수,5(2-1),2루수 땅볼,무사 0:0,1사 0:0,0.87,0.555,-0.258,52.6%,-0.022,2021-05-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
670849,,오승환,2 손아섭,5(2-2),2루수 땅볼,무사 1:4,1사 1:4,0.75,0.555,-0.258,98.6%,-0.019,2021-05-07
670850,,오승환,3 전준우,1(0-0),유격수 땅볼,1사 1:4,2사 1:4,0.40,0.297,-0.180,99.6%,-0.010,2021-05-07
670851,,오승환,4 이대호,2(1-0),우익수 안타,2사 1:4,2사 1루 1:4,0.14,0.117,0.134,98.8%,0.008,2021-05-07
670852,,오승환,5 한동희,,*폭투,2사 1루 1:4,2사 2루 1:4,0.39,0.251,0.093,98.7%,0.001,2021-05-07
